In [1]:
##!pip3 install torch

In [2]:
##!pip3 install numpy

In [3]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

import numpy as np
import scipy.sparse as sp
from topic_modeling import VAE, Encoder, Decoder, SparseDataset

In [4]:
ensure_pyterrier_is_loaded()
tira = Client()

# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
# hyperparameters

cuda = False
device = torch.device("cuda" if cuda else "cpu")

epochs = 10
batch_size = 100
learning_rate = 1e-3

In [6]:
""" # bow
# bow = {'O02-2002' : {'i': 2, 'mixtur': 1, 'adopt': 1, 'exampl': 1, 'featur': 2, 'especi': 1, 'classif': 1, 'requir': 1, 'gener': 1, 'measur': 2, 'idf': 1, 'characterist': 1, 'valu': 2, 'languag': 1, 'vector': 3, 'environ': 1, 'data': 1, 'turn': 1}}


bow = {}
for docid in range(doi.getNumberOfDocuments()):
  for posting in  di.getPostings(doi.getDocumentEntry(docid)):
    docno = meta.getItem("docno", docid)
    if not docno in bow:
      bow[docno]= {}
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    bow[docno][lee.getKey()] = posting.getFrequency()

print(bow["O02-2002"]) """

' # bow\n# bow = {\'O02-2002\' : {\'i\': 2, \'mixtur\': 1, \'adopt\': 1, \'exampl\': 1, \'featur\': 2, \'especi\': 1, \'classif\': 1, \'requir\': 1, \'gener\': 1, \'measur\': 2, \'idf\': 1, \'characterist\': 1, \'valu\': 2, \'languag\': 1, \'vector\': 3, \'environ\': 1, \'data\': 1, \'turn\': 1}}\n\n\nbow = {}\nfor docid in range(doi.getNumberOfDocuments()):\n  for posting in  di.getPostings(doi.getDocumentEntry(docid)):\n    docno = meta.getItem("docno", docid)\n    if not docno in bow:\n      bow[docno]= {}\n    termid = posting.getId()\n    lee = lex.getLexiconEntry(termid)\n    bow[docno][lee.getKey()] = posting.getFrequency()\n\nprint(bow["O02-2002"]) '

In [7]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
meta = index.getMetaIndex()

bow_sparse = sp.dok_matrix((doi.getNumberOfDocuments(),len(lex)), dtype=np.int8)

for docid in range(doi.getNumberOfDocuments()):
  docno = meta.getItem("docno", docid)
  # if docid == 0:
    # print(docno)
  for posting in  di.getPostings(doi.getDocumentEntry(docid)):
    if posting.getFrequency() > 0:
      termid = posting.getId()
      lee = lex.getLexiconEntry(termid)
      # if docid == 0:
        # print(f"{lee.getKey()} , {termid}")
      bow_sparse[docid, termid] = posting.getFrequency()
      # if docid == 0:
        # print(f"{docid}")
        # print(f"{bow_sparse[docid, termid]}")

bow_sparse = bow_sparse.tocsr()

In [ ]:
#print(bow_sparse[31])
print(bow_sparse.shape)
print(bow_sparse[31].todense().item(117))


(126958, 97223)
7


In [ ]:
dim = bow_sparse.shape
device = torch.device(device)
indptr = torch.tensor(bow_sparse.indptr, dtype=torch.int64, device=device)
indices = torch.tensor(bow_sparse.indices, dtype=torch.int64, device=device)
data = torch.tensor(bow_sparse.data, dtype=torch.int8, device=device)

print(dim[0])

idx = 0

obs = torch.zeros((dim[1],), dtype=torch.int8, device=device)
print(obs)
print(len(obs))
print(obs[0])
ind1,ind2 = indptr[idx],indptr[idx+1]
obs[indices[ind1:ind2]] = data[ind1:ind2]
print(obs.shape)
print(obs)

126958
tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int8)
97223
tensor(0, dtype=torch.int8)
torch.Size([97223])
tensor([2, 1, 1,  ..., 0, 0, 0], dtype=torch.int8)


In [ ]:
train_dataset = SparseDataset(bow_sparse, device)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(train_dataset.__getitem__(0))

tensor([2., 1., 1.,  ..., 0., 0., 0.])


In [ ]:
""" for batch, x in enumerate(train_loader):
    print(batch)
    for doc in x:
        for freq in doc:
            if freq > 0:
                print(freq)
                break;
    break """

' for batch, x in enumerate(train_loader):\n    print(batch)\n    for doc in x:\n        for freq in doc:\n            if freq > 0:\n                print(freq)\n                break;\n    break '

In [ ]:
# more hyperparameters

input_dim  = len(lex)
hidden_dim = 800
latent_dim = 200

In [ ]:
encoder = Encoder(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = input_dim)

vae = VAE(encoder=encoder, decoder=decoder, device=device).to(device)

In [ ]:
print(vae)

VAE(
  (encoder): Encoder(
    (FC_input): Linear(in_features=97223, out_features=800, bias=True)
    (FC_input2): Linear(in_features=800, out_features=800, bias=True)
    (FC_mean): Linear(in_features=800, out_features=200, bias=True)
    (FC_var): Linear(in_features=800, out_features=200, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
  (decoder): Decoder(
    (FC_hidden): Linear(in_features=200, out_features=800, bias=True)
    (FC_hidden2): Linear(in_features=800, out_features=800, bias=True)
    (FC_output): Linear(in_features=800, out_features=97223, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
)


In [ ]:
BCE_loss = nn.BCELoss()

def loss_function(x, x_hat, mean, log_var):
    logsoftmax = torch.log_softmax(x_hat,dim=1)
    rec_loss = -1.0 * torch.sum(x*logsoftmax)

    kl_div = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return rec_loss + kl_div

optimizer = Adam(vae.parameters(), lr=learning_rate)

In [ ]:
print("VAE training:")
vae.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, x in enumerate(train_loader):
        x = x.view(batch_size, input_dim)
        x = x.to(device)

        optimizer.zero_grad()

        x_hat, mean, log_var = vae(x)
        
        loss = loss_function(x, x_hat, mean, log_var)
        
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch + 1} :\tAverage Loss: {overall_loss / (batch_idx*batch_size)}")
    
print("VAE training done")

VAE training:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
hi1
torch.float32
torch.float32
hi2
hi3
hi4
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
hi1
torch.float32
torch.float32
hi2
hi3
hi4
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
hi1
torch.float32
torch.float32
hi2
hi3
hi4


KeyboardInterrupt: 